In [ ]:
from PosSelect_Functions_Old import *
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import pandas as pd
import numpy as np
import copy
import seaborn as sns
from scipy.stats import mannwhitneyu as mwu
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import fdrcorrection
from scipy.stats import wilcoxon
from scipy.optimize import curve_fit
from scipy.stats import fisher_exact
from scipy.stats import norm
from collections import Counter
from scipy.stats import binomtest
import os
import matplotlib
hfont = {'fontname':'Arial'}
plt.rcParams["font.family"] = "Arial"

#Code borrowed heavily from here: https://stackoverflow.com/questions/62375034/find-non-overlapping-area-between-two-kde-plots
plt.rcParams.update(
    {"text.usetex": False}
)

def parse_table(i):
    return [int(x) for x in i.replace("[", "").replace("]", "").split(",")]

d_abrev = {"LiangSteinNeuron":"FC exc. neur.", "FetalChondrocytes":"F chond.", "SertoliMale":"FG sertoli", "preGC_IIaFemale":"FG preGC IIa",\
          "NeuralFemale":"FG neur.", "FetalGonadImmuneFemale":"FG immune", "VIP":"AC VIP inh. neur.", "LiangSteinProgenitor":"FC prog.",\
          "AdultHeartVentricularCardiomyocyte":"AH cardiomyo.", "AdultLoopOfHenle":"AK loop of henle", "FetalBrainNeurGlioblast_CB_VZ":"FCB glioblast",\
         "AdultProximalTubule":"AK prox. tub.", "FetalLeydigMale":"FG leydig", "SST":"AC SST inh neur.", "KosoyRoussosControlMicroglia":"AC microglia",\
         "FetalBrainFloorPlate":"FB fl. plate", "FetalArterialECs":"FH endoth.", "ASCT":"AC astro.", "FetalBrainCOP":"FB COP",\
         "AMY":"AA neur.", "PVALB":"AC PVALB inh neur.", "ITL23":"AC L2-3 IT neur.", "FetalBrainNeurCB_GNP_IPC_1":"FB inter. prog.", "FetalBrainNeurDAergic":"FB DA neur.",\
          "OGC":"AC Oligo.", "D1Pu":"AP D1 inh neur.", "FetalBrainNeurSerotonergic":"FB 5-HT neur.", "FetalBrainNeurDRG_2":"FS DRG neur.",\
          "FetalHeartPericytes":"FH peri.", "FetalHeartEndocardium":"FH endocard.", "FetalHeartCardiacFibroblasts":"FH fibro.", "FetalBrainNeurPurkinje_6":"FCB Purk. inh neur.",\
          "AdultHeartSmoothMuscle":"AH smooth musc.", "FetalBrainRoofPlate":"FB ro. plate"}

In [ ]:
#Read in
v = pd.read_csv("RHCTAG_Humreffed_Positions_Filt_DL_Data.txt", sep = "\t")
#v = pd.read_csv("RHCTAL_Chpreffed_Positions_Filt_DL_Data.txt", sep = "\t")

In [ ]:
#Restrict to group1 or group2
group1 = ["KosoyRoussosControlMicroglia", "LiangSteinNeuron", "AdultHeartVentricularCardiomyocyte", "AdultProximalTubule", "FetalArterialECs", "FetalChondrocytes", "SertoliMale", "ASCT"]
group2 = ["FetalGonadImmuneFemale", "ITL23", "AdultHeartSmoothMuscle", "AdultLoopOfHenle", "FetalHeartPericytes", "FetalHeartCardiacFibroblasts", "preGC_IIaFemale", "FetalBrainCOP"]

keep = []
for i in v.columns:
    if "logfc" in i and "abs" not in i and i.replace("logfc ", "") in group2:
        keep.append(i)
v2 = v[["Position"] + keep]
v2 = -v2.set_index("Position")

keep = []
for i in v.columns:
    if "allele1_pred_counts" in i and "abs" not in i and i.replace("allele1_pred_counts ", "") in group2:
        keep.append(i)
v3 = v[["Position"] + keep]
v3 = v3.set_index("Position")

In [ ]:
#Read in the counts data
dfn = pd.read_csv("CTAGL_Split/RHCTAG_Humreffed_Data_Filt.txt", sep = "\t")
#dfn = pd.read_csv("CTAGL_Split/RHCTAL_Chpreffed_Data_Filt.txt", sep = "\t")
out = []
for index, row in dfn.iterrows():
    for i in row["Positions"].split(";"):
        out.append([i, row["Position"], row["Cell type"]])
v2 = v2.join(pd.DataFrame(out).set_index(0))
v3 = v3.join(pd.DataFrame(out).set_index(0))
v3

In [ ]:
#Look at potential example genes
x = pd.read_csv("20250618-public-4.0.4-FaA5o2-hg38-GOMolecularFunction-GO_0072509-gene.txt", sep = "\t", skiprows = 1, header = None)
dfn[(dfn["NearestGene"].isin(x[0])) & (dfn["Cell type"].isin(["LiangSteinNeuron", "ITL23", "LiangSteinNeuron;ITL23", "ITL23;LiangSteinNeuron"]))].sort_values("Summed EE_Dif")

In [ ]:
l23u[l23u["Gene"].isin(genes)]

In [ ]:
dfn[dfn["NearestGene"] == "CACNA2D3"]

In [ ]:
l23 = pd.read_csv("DESeq2_L2-3_IT_Human_Chimp.txt", sep = "\t")
l23 = l23.dropna()
l23 = l23[l23["padj"] < 0.05]
l23d = l23[l23["log2FoldChange"] < 0]
l23u = l23[l23["log2FoldChange"] > 0]
down = l23d[l23d["Gene"].isin(genes)].shape[0]
up = l23u[l23u["Gene"].isin(genes)].shape[0]

from scipy.stats import binomtest
binomtest(down, up + down, p = l23d.shape[0]/(l23d.shape[0] + l23u.shape[0]))

In [ ]:
#Pick a particular RAG/RAL
v2p = v2[v2[1] == "chr3:54170706"]
v3p = v3[v3[1] == "chr3:54170706"]

out = []
for index, row in v2p.iterrows():
    for i in row.index:
        if i != 1 and i != 2:
            out.append([index, d_abrev[i.replace("logfc ", "")], row[i]])

In [ ]:
#Make plot
v2plot = pd.DataFrame(out).sort_values(1)
v2plot.columns = ["Position", "Cell type", "Log fold-change"]

plot = pd.DataFrame()
for i in np.unique(v2plot["Position"]):
    v2pp = v2plot[v2plot["Position"] == i]
    v2pp = v2pp.sort_values("Log fold-change", ascending = False)
    plot = pd.concat([plot, v2pp])

In [ ]:
colors = [matplotlib.colors.to_hex(x) for x in sns.color_palette("colorblind")]
palette_g1 = {}
for i in range(len(group1)):
    palette_g1[d_abrev[group1[i]]] = colors[i]

palette_g2 = {}
for i in range(len(group2)):
    palette_g2[d_abrev[group2[i]]] = colors[i]


In [ ]:
#Set focal cell type to make magenta
focal_ct = d_abrev["LiangSteinNeuron"]
for i in group1:
    palette_g1[d_abrev[i]] = "grey"
palette_g1[focal_ct] = "magenta"

focal_ct = d_abrev["ITL23"]
for i in group2:
    palette_g2[d_abrev[i]] = "grey"
palette_g2[focal_ct] = "magenta"

In [ ]:
#Make plot
fig, ax = plt.subplots(figsize = (6, 4))
sns.barplot(data = v2plot, x = "Position", y = "Log fold-change", hue = "Cell type", palette = palette_g2)
plt.legend([], [], frameon = False)
#plt.title("HAR near NR2F2")
plt.xticks(size = 14, rotation = 0)
plt.yticks(size = 14)

plt.ylabel("Log$_{2}$ fold-change", size = 18)
plt.xlabel("Genomic position", size = 18)
plt.title("hRAG near $\it{CACNA2D3}$", size = 18)

In [ ]:
fig, ax = plt.subplots(figsize = (6, 4))
sns.barplot(data = v2plot, x = "Position", y = "Log fold-change", hue = "Cell type", palette = palette_g2)
plt.legend([], [], frameon = False)
#plt.title("HAR near NR2F2")
plt.xticks(size = 11, rotation = 0)
plt.yticks(size = 11)

plt.ylabel("Log$_{2}$ fold-change", size = 14)
plt.xlabel("Genomic position", size = 14)
plt.title("hRAG near $\it{CACNA2D3}$", size = 14)

In [ ]:
#Plot number in each cell type
dfn = pd.read_csv("RHCTAG_Data_Filt.txt", sep = "\t")
x = []
for index, row in dfn.iterrows():
    x = x + row['Cell type'].split(";")
c = Counter(x)
out = []
for key in c.keys():
    out.append([d_abrev[key], c[key], "Human", "hRAG"])
    
dfn = pd.read_csv("RHCTAL_Data_Filt.txt", sep = "\t")
x = []
for index, row in dfn.iterrows():
    x = x + row['Cell type'].split(";")
c = Counter(x)
for key in c.keys():
    out.append([d_abrev[key], c[key], "Human", "hRAL"])
    
dfn = pd.read_csv("RCCTAG_Data_Filt.txt", sep = "\t")
x = []
for index, row in dfn.iterrows():
    x = x + row['Cell type'].split(";")
c = Counter(x)
for key in c.keys():
    out.append([d_abrev[key], c[key], "Chimp", "cRAG"])
    
dfn = pd.read_csv("RCCTAL_Data_Filt.txt", sep = "\t")
x = []
for index, row in dfn.iterrows():
    x = x + row['Cell type'].split(";")
c = Counter(x)
for key in c.keys():
    out.append([d_abrev[key], c[key], "Chimp", "cRAL"])

dfp = pd.DataFrame(out)
dfp.columns = ["Cell type", "Number elements identified", "Species", "Element type"]



In [ ]:
palette = {"hRAG":"#E21814", "hRAL":"#5A0908", "cRAG":"#2142EA", "cRAL":"#11237C"}
pal_chp = {"cRAG":"#2142EA", "cRAL":"#11237C"}

fig, ax = plt.subplots(figsize = (10, 4))
dfp2 = dfp.sort_values("Cell type").sort_values("Element type", ascending = False)
sns.set(font_scale = 1.5)
sns.set_style("white")
sns.barplot(data = dfp2, x = "Cell type", y = "Number elements identified", hue = "Element type", palette = palette)
plt.xticks(rotation = 90)
plt.title("Reinforcing cell type-specific changes in CA across cell types")